In [1]:
import pandas as pd
from src.data.load_data import load_price_data, load_event_data

prices = load_price_data("data/raw/BrentOilPrices.csv")
events = load_event_data("data/events/oil_market_events.csv")

events.columns = events.columns.str.lower()
events["date"] = pd.to_datetime(events["date"])

WINDOW = 30  # days

results = []

for _, event in events.iterrows():
    start = event["date"] - pd.Timedelta(days=WINDOW)
    end = event["date"] + pd.Timedelta(days=WINDOW)

    before = prices[
        (prices["Date"] >= start) & (prices["Date"] < event["date"])
    ]["Price"].mean()

    after = prices[
        (prices["Date"] > event["date"]) & (prices["Date"] <= end)
    ]["Price"].mean()

    impact_pct = ((after - before) / before) * 100 if before else None

    results.append({
        "event": event["event"],
        "category": event["category"],
        "date": event["date"],
        "avg_price_before": before,
        "avg_price_after": after,
        "impact_pct": impact_pct
    })

impact_df = pd.DataFrame(results)
impact_df.sort_values("impact_pct", ascending=False)


ModuleNotFoundError: No module named 'src'